In [2]:
import pandas as pd
import numpy as np
import torch
import os
import sys
sys.path.append("/home/docker_current/py_files/MLCLIP_exp")
sys.path.append("/home/docker_current/py_files/finetune_exp/metaclip_arch")
sys.path.append("/home/docker_current/py_files/utils")
from MLCLIP_utils import get_text_encode_model, get_image_encode_model

device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [3]:
from MLCLIP import MLCLIPPredictor
predictor = MLCLIPPredictor(device='cuda:1')

# metaclip_model = MetaCLIP(ratio=0.5)
# path_weights = "./weights/ruclip_v5.pth"

Some weights of the model checkpoint at xlm-roberta-large were not used when initializing XLMRobertaModel: ['lm_head.layer_norm.bias', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.bias']
- This IS expected if you are initializing XLMRobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
df_test = pd.read_csv("/home/docker_current/datasets/test.csv")
df_test['text_features'] = None
df_test['object_img'] = None

In [15]:
df_train = pd.read_csv("/home/docker_current/datasets/train.csv")

name_emb_file = "/home/docker_current/py_files/embeddings/Roberta_Large_Vit_L_14.npy"

with open(name_emb_file, 'rb') as f:
    emb_text = np.load(f)
    emb_image = np.load(f)

df_train['emb_text'] = [i for i in emb_text]

df_train.head()

,id,description,object_img,emb_text
0,520,Фотография. Г. Пермь. Здание горисполкома. ПО...,799,"[-0.24118489, 0.25865138, -0.14606075, 1.22039..."
1,1817,Фотонегатив пленочный. Труппа театра «У моста»...,854,"[-0.12055853, 0.15714812, 0.032904923, 0.78152..."
2,188,"Изразец гладкий расписной ""пермский""- карниз А...",1794,"[0.30912644, 0.8710553, -0.00015427265, -0.003..."
3,1472,Фотонегатив пленочный. Серебряная чаша из Хоре...,526,"[-0.11504096, -0.2660843, 0.25976348, 0.097250..."
4,1664,Фотонегатив стеклянный. Этнография коми-пермяк...,244,"[-0.19554895, 0.33908838, 0.0039799055, -0.282..."


In [19]:
test_images = [i for i in df_train['object_img']]
t_emb = [i for i in df_train['emb_text']]
test_embed = {i:j for i, j in zip(test_images, t_emb)}

In [23]:
from tqdm.notebook import tqdm

for ind_text in tqdm(range(len(df_test))):
    text = df_test['description'][ind_text]
    text_features = predictor.inference_text(text)
    text_features = torch.Tensor(text_features)
    # with torch.no_grad():
    #     text_features = metaclip_model.encode_image(text_features.to(device)).cpu().detach().numpy()
    df_test['text_features'][ind_text] = text_features

  0%|          | 0/900 [00:00<?, ?it/s]

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':


In [24]:
# from tqdm.notebook import tqdm

# for name_image in tqdm(test_embed.keys()):
#     text= df_train[df_train['object_img'] == name_image]['description'].values[0]
#     features = predictor.inference_text(text)
#     features = torch.Tensor(features)
#     # with torch.no_grad():
#     #     features = metaclip_model.encode_image(image_features.to(device)).cpu().detach().numpy()
#     test_embed[name_image] = features

In [25]:
from copy import deepcopy
copy_test_embed = deepcopy(test_embed)

In [26]:
cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)

def get_similarity(image_emb, text_emb):
    sim =cos(torch.Tensor(image_emb), 
                        torch.Tensor(text_emb))
    return sim

In [27]:
preds = []

for ind_text in tqdm(range(len(df_test))):

    sims = []
    for image_name in copy_test_embed.keys():
        sim = get_similarity(copy_test_embed[image_name], 
                                    df_test['text_features'][ind_text])
        sims.append(sim)

    sims = np.array(sims)
    ind_max = np.argmax(sims)
    match_image = list(copy_test_embed.keys())[ind_max]
    preds.append(match_image)
    # del copy_test_embed[match_image]

  0%|          | 0/900 [00:00<?, ?it/s]

In [28]:
submit = pd.read_csv('/home/docker_current/py_files/sample_solution.csv')
submit['object_img'] = preds
submit['object_img'] = submit['object_img'].astype(np.int64)
submit.to_csv('./submits/dummy_3_repeat_only_text.csv', index=False)

In [29]:
len(copy_test_embed.keys()), len(set(list(submit['object_img'])))

(2098, 668)